# <center> Simulation Video

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import matplotlib.animation as animation
from tqdm import tqdm
import pickle
import json

# ==============================================================================
# GLOBAL PARAMETERS
# ==============================================================================

# Video parameters
N_FRAMES = 1000  # Number of frames to render (adjust this to control video length)
                 # Set to None to render all timesteps
FPS = 20         # Frames per second for the video

# Edge width metric choice
EDGE_METRIC = 'weight'  # Options: 'weight' (metric tons) or 'value' (USD)
                        # Controls both edge width and ranking order

# Ship type colors
SHIP_TYPE_COLORS = {
    'tanker': {'name': 'Tanker', 'color': '#8B4513'},       # Brown
    'bulk carrier': {'name': 'Bulk Carrier', 'color': '#FFD700'},  # Gold
    'cargo ship': {'name': 'Cargo Ship', 'color': '#4169E1'}  # Royal Blue
}

# Default color for ships with no type data
DEFAULT_SHIP_COLOR = 'gray'

In [29]:
# ==============================================================================
# LOAD DATA
# ==============================================================================
print("Loading network and simulation data...")

# Load HS codes mapping
with open('../../data/hs_codes_mapping.json', 'r') as f:
    hs_codes_mapping = json.load(f)

print(f"✓ Loaded HS codes mapping: {len(hs_codes_mapping)} commodities")

# Load network
with open('../part_3_network_extraction/network_outputs/network_contraction_hierarchies.gpickle', 'rb') as f:
    G_ch = pickle.load(f)

print(f"✓ Network loaded: {G_ch.number_of_nodes()} nodes, {G_ch.number_of_edges()} edges")

# Load ship locations
with open('../part_4_simulation/simulation_output_data/simulation_ship_location.json', 'r') as f:
    ship_locations = json.load(f)

print(f"✓ Ship locations loaded: {len(ship_locations)} timesteps")

# Load ship data
ship_data_df = pd.read_csv('../part_4_simulation/simulation_output_data/simulation_ship_data.csv')
print(f"✓ Ship data loaded: {len(ship_data_df)} ships")

# Extract HS codes present in data
hs_codes_in_data = []
for col in ship_data_df.columns:
    if col.startswith('cargo_hs') and col.endswith('_weight'):
        hs_code = int(col.replace('cargo_hs', '').replace('_weight', ''))
        hs_codes_in_data.append(hs_code)

print(f"✓ Found {len(hs_codes_in_data)} HS codes in ship data: {sorted(hs_codes_in_data)}")

# Create mappings for ship data
ship_weight_map = dict(zip(ship_data_df['ship_id'], ship_data_df['cargo_total_weight']))
ship_value_map = dict(zip(ship_data_df['ship_id'], ship_data_df['cargo_total_value']))
ship_type_map = dict(zip(ship_data_df['ship_id'], ship_data_df['ship_type']))

# Create ship color map based on ship type
ship_color_map = {}
for ship_id, ship_type in ship_type_map.items():
    if ship_type in SHIP_TYPE_COLORS:
        ship_color_map[ship_id] = SHIP_TYPE_COLORS[ship_type]['color']
    else:
        ship_color_map[ship_id] = DEFAULT_SHIP_COLOR

# Calculate cargo statistics for scaling (both weight and value)
min_weight = ship_data_df['cargo_total_weight'].min()
max_weight = ship_data_df['cargo_total_weight'].max()
mean_weight = ship_data_df['cargo_total_weight'].mean()

min_value = ship_data_df['cargo_total_value'].min()
max_value = ship_data_df['cargo_total_value'].max()
mean_value = ship_data_df['cargo_total_value'].mean()

print(f"\nCargo weight statistics:")
print(f"  Min: {min_weight:,.0f} tons")
print(f"  Max: {max_weight:,.0f} tons")
print(f"  Mean: {mean_weight:,.0f} tons")

print(f"\nCargo value statistics:")
print(f"  Min: ${min_value:,.0f}")
print(f"  Max: ${max_value:,.0f}")
print(f"  Mean: ${mean_value:,.0f}")

# Show ship type distribution
print(f"\nShip type distribution:")
for ship_type_key, info in SHIP_TYPE_COLORS.items():
    count = sum(1 for st in ship_type_map.values() if st == ship_type_key)
    percentage = 100 * count / len(ship_data_df) if len(ship_data_df) > 0 else 0
    print(f"  {info['name']}: {count} ships ({percentage:.1f}%) - Color: {info['color']}")

# Create HS code to weight/value mappings for each ship
ship_hs_weights = {}  # ship_id -> {hs_code: weight}
ship_hs_values = {}   # ship_id -> {hs_code: value}

for idx, row in ship_data_df.iterrows():
    ship_id = row['ship_id']
    ship_hs_weights[ship_id] = {}
    ship_hs_values[ship_id] = {}
    
    for hs_code in hs_codes_in_data:
        weight_col = f'cargo_hs{hs_code}_weight'
        value_col = f'cargo_hs{hs_code}_value'
        
        if weight_col in row and value_col in row:
            ship_hs_weights[ship_id][hs_code] = row[weight_col]
            ship_hs_values[ship_id][hs_code] = row[value_col]

# Load port occupancy data
port_occupancy_df = pd.read_csv('../part_4_simulation/simulation_output_data/simulation_port_occupancy.csv')
print(f"\n✓ Port occupancy loaded: {len(port_occupancy_df)} records")

# Create port occupancy lookup: timestep -> {port_name: (num_ships, capacity)}
port_occupancy_by_timestep = {}
for _, row in port_occupancy_df.iterrows():
    timestep = int(row['timestep'])
    port_name = row['port_name']
    num_ships = int(row['num_ships'])
    capacity = int(row['capacity'])
    
    if timestep not in port_occupancy_by_timestep:
        port_occupancy_by_timestep[timestep] = {}
    port_occupancy_by_timestep[timestep][port_name] = (num_ships, capacity)

print(f"✓ Port occupancy indexed by {len(port_occupancy_by_timestep)} timesteps")

# Load world map for visualization
world = gpd.read_file('https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_0_countries.zip')
print("✓ World map loaded")

print("\n" + "="*70)
print(f"EDGE METRIC: {EDGE_METRIC.upper()}")
print(f"  Edge widths will represent cumulative {EDGE_METRIC}")
print(f"  Rankings will be ordered by {EDGE_METRIC}")
print("="*70)

Loading network and simulation data...
✓ Loaded HS codes mapping: 97 commodities
✓ Network loaded: 328 nodes, 526 edges
✓ Ship locations loaded: 8760 timesteps
✓ Ship data loaded: 8681 ships
✓ Found 96 HS codes in ship data: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97]

Cargo weight statistics:
  Min: 10,485 tons
  Max: 177,213 tons
  Mean: 57,675 tons

Cargo value statistics:
  Min: $2,304,624
  Max: $21,962,329,396
  Mean: $77,295,939

Ship type distribution:
  Tanker: 4491 ships (51.7%) - Color: #8B4513
  Bulk Carrier: 2112 ships (24.3%) - Color: #FFD700
  Cargo Ship: 2078 ships (23.9%) - Color: #4169E1

✓ Port occupancy loaded: 161814 records
✓ Port occupan

In [30]:
# ==============================================================================
# CREATE ANIMATED VIDEO VISUALIZATION
# ==============================================================================
print("Creating animated video visualization...")
print("This may take several minutes...\n")

# Scaling parameters (defined early so they can be used in debug prints)
MIN_SHIP_SIZE = 10
MAX_SHIP_SIZE = 100
MIN_EDGE_WIDTH = 0.3
MAX_EDGE_WIDTH = 10.0
EDGE_NO_TRAFFIC_ALPHA = 0.1
EDGE_WITH_TRAFFIC_ALPHA = 0.25

# Convert node ID from string to appropriate type (int or str)
def convert_node_id(node):
    '''Convert node ID to appropriate type (int or str)'''
    node_str = str(node).strip()
    if node_str.startswith('port_'):
        return node_str
    try:
        return int(node_str)
    except (ValueError, TypeError):
        return node_str

# Build canonical edge lookup
print("Building canonical edge lookup...")
canonical_edges = {}
for u, v in G_ch.edges():
    canonical_edges[(u, v)] = (u, v)
    canonical_edges[(v, u)] = (u, v)

print(f"Canonical edge lookup built: {len(canonical_edges)} entries ({G_ch.number_of_edges()} unique edges)\n")

# Create port name to node mapping
port_name_to_node = {}
for node in G_ch.nodes():
    if G_ch.nodes[node].get('source') == 'port':
        port_name = G_ch.nodes[node].get('portName')
        if port_name:
            port_name_to_node[port_name] = node

# Get network extent
lons = [G_ch.nodes[n]['lon'] for n in G_ch.nodes()]
lats = [G_ch.nodes[n]['lat'] for n in G_ch.nodes()]
lon_min, lon_max = min(lons) - 1, max(lons) + 1
lat_min, lat_max = min(lats) - 1, max(lats) + 1

# Convert ship_locations dict to sorted list
all_timesteps = sorted([(float(ts), data) for ts, data in ship_locations.items()])

# Limit to N_FRAMES if specified
if N_FRAMES is not None and N_FRAMES < len(all_timesteps):
    timesteps = all_timesteps[:N_FRAMES]
    print(f"Limiting animation to first {N_FRAMES} frames (out of {len(all_timesteps)} total)")
else:
    timesteps = all_timesteps
    print(f"Creating animation with all {len(timesteps)} frames")

video_duration = len(timesteps) / FPS
print(f"Video duration: {video_duration:.1f} seconds at {FPS} fps")
print(f"Timestep range: Day {timesteps[0][0]:.1f} to Day {timesteps[-1][0]:.1f}\n")

# Helper function for port colors
def get_port_color(num_ships, capacity):
    '''Return port color based on occupancy percentage'''
    if num_ships == 0:
        return 'green'
    occupancy_pct = num_ships / capacity if capacity > 0 else 0
    if occupancy_pct < 0.5:
        return 'green'
    elif occupancy_pct < 1.0:
        return 'orange'
    else:
        return 'red'

# ==============================================================================
# PRE-COMPUTE EDGE CARGO AND COMPLETED SHIPS
# ==============================================================================
print("Pre-computing edge cargo accumulation and completed ships tracking...")

edge_cargo_by_frame = []
completed_ships_by_frame = []
completed_cargo_by_frame = []

cumulative_edge_cargo = {}
ship_edge_history = {}
completed_ships = set()
completed_hs_weights = {}
completed_hs_values = {}

for frame_idx, (day, locations_data) in enumerate(tqdm(timesteps, desc="Computing data")):
    # DEBUG: Print header for first 50 frames
    if frame_idx < 50:
        print(f"\n{'='*80}")
        print(f"FRAME {frame_idx} (Day {day:.1f})")
        print(f"{'='*80}")
    
    # Track ships entering edges
    for ship_id, loc in locations_data.items():
        if loc['status'] == 'active':
            edge = loc['edge']
            node1 = convert_node_id(edge[0])
            node2 = convert_node_id(edge[1])
            edge_key = canonical_edges.get((node1, node2))
            
            if edge_key is None:
                continue
            
            ship_id_int = int(ship_id)
            
            if ship_id_int not in ship_edge_history:
                ship_edge_history[ship_id_int] = set()
            
            if edge_key not in ship_edge_history[ship_id_int]:
                ship_edge_history[ship_id_int].add(edge_key)
                
                if edge_key not in cumulative_edge_cargo:
                    cumulative_edge_cargo[edge_key] = {'weight': 0.0, 'value': 0.0}
                
                # DEBUG: Print ship entering new edge for first 50 frames
                if frame_idx < 50:
                    prev_weight = cumulative_edge_cargo[edge_key]['weight']
                    prev_value = cumulative_edge_cargo[edge_key]['value']
                    ship_weight = ship_weight_map.get(ship_id_int, 0)
                    ship_value = ship_value_map.get(ship_id_int, 0)
                    
                    print(f"\nShip {ship_id_int} entering edge {edge_key}")
                    print(f"  Ship weight: {ship_weight:,.0f} tons, Ship value: ${ship_value:,.0f}")
                    print(f"  Edge weight BEFORE: {prev_weight:,.0f} tons")
                    print(f"  Edge value BEFORE:  ${prev_value:,.0f}")
                
                if ship_id_int in ship_weight_map:
                    cumulative_edge_cargo[edge_key]['weight'] += ship_weight_map[ship_id_int]
                if ship_id_int in ship_value_map:
                    cumulative_edge_cargo[edge_key]['value'] += ship_value_map[ship_id_int]
                
                # DEBUG: Print updated values for first 50 frames
                if frame_idx < 50:
                    new_weight = cumulative_edge_cargo[edge_key]['weight']
                    new_value = cumulative_edge_cargo[edge_key]['value']
                    print(f"  Edge weight AFTER:  {new_weight:,.0f} tons (+{new_weight - prev_weight:,.0f})")
                    print(f"  Edge value AFTER:   ${new_value:,.0f} (+${new_value - prev_value:,.0f})")
    
    # Track completed ships
    if frame_idx > 0:
        prev_day, prev_locations = timesteps[frame_idx - 1]
        
        for ship_id, loc in prev_locations.items():
            if loc['status'] == 'unloading':
                ship_id_int = int(ship_id)
                if ship_id not in locations_data and ship_id_int not in completed_ships:
                    completed_ships.add(ship_id_int)
                    
                    if ship_id_int in ship_hs_weights:
                        for hs_code, weight in ship_hs_weights[ship_id_int].items():
                            if hs_code not in completed_hs_weights:
                                completed_hs_weights[hs_code] = 0.0
                            completed_hs_weights[hs_code] += weight
                    
                    if ship_id_int in ship_hs_values:
                        for hs_code, value in ship_hs_values[ship_id_int].items():
                            if hs_code not in completed_hs_values:
                                completed_hs_values[hs_code] = 0.0
                            completed_hs_values[hs_code] += value
    
    # Store DEEP COPY snapshots for this frame
    # We need deep copies because the nested dicts will continue to be modified
    frame_snapshot = {}
    for edge_key, cargo_dict in cumulative_edge_cargo.items():
        frame_snapshot[edge_key] = {'weight': cargo_dict['weight'], 'value': cargo_dict['value']}
    
    edge_cargo_by_frame.append(frame_snapshot)
    completed_ships_by_frame.append(set(completed_ships))
    completed_cargo_by_frame.append({
        'weight': sum(completed_hs_weights.values()),
        'value': sum(completed_hs_values.values()),
        'hs_weights': dict(completed_hs_weights),
        'hs_values': dict(completed_hs_values)
    })

# Calculate maximum edge cargo at the END of the video (final frame)
# This is used to scale all edge widths throughout the animation
final_frame_edge_cargo = edge_cargo_by_frame[-1]
if final_frame_edge_cargo:
    max_edge_weight = max(e['weight'] for e in final_frame_edge_cargo.values())
    max_edge_value = max(e['value'] for e in final_frame_edge_cargo.values())
else:
    max_edge_weight = 0
    max_edge_value = 0

print(f"\n{'='*80}")
print(f"FINAL MAX VALUES (from frame {len(timesteps)-1}):")
print(f"  Max edge weight: {max_edge_weight:,.0f} tons")
print(f"  Max edge value:  ${max_edge_value:,.0f}")
print(f"{'='*80}\n")

# Now print edge widths for first 50 frames
print("\nEDGE WIDTH CALCULATIONS FOR FIRST 50 FRAMES:")
print(f"Using EDGE_METRIC = '{EDGE_METRIC}'")
print(f"MIN_EDGE_WIDTH = {MIN_EDGE_WIDTH}, MAX_EDGE_WIDTH = {MAX_EDGE_WIDTH}")

for frame_idx in range(min(50, len(edge_cargo_by_frame))):
    print(f"\n{'='*80}")
    print(f"FRAME {frame_idx} EDGE WIDTHS:")
    print(f"{'='*80}")
    
    # Use the SNAPSHOT from this specific frame
    frame_edge_cargo = edge_cargo_by_frame[frame_idx]
    max_edge_metric = max_edge_weight if EDGE_METRIC == 'weight' else max_edge_value
    
    # Get edges with traffic, sorted by metric
    edges_with_traffic = [(edge_key, data) for edge_key, data in frame_edge_cargo.items() 
                          if data.get(EDGE_METRIC, 0) > 0]
    edges_with_traffic.sort(key=lambda x: x[1].get(EDGE_METRIC, 0), reverse=True)
    
    if edges_with_traffic:
        print(f"Edges with traffic: {len(edges_with_traffic)}")
        print(f"Max edge metric (final frame): {max_edge_metric:,.0f}")
        print(f"\nTop 5 edges by {EDGE_METRIC}:")
        
        for i, (edge_key, edge_data) in enumerate(edges_with_traffic[:5]):
            metric_value = edge_data.get(EDGE_METRIC, 0)
            
            # Calculate width (same formula as in animate function)
            if max_edge_metric > 0:
                normalized = metric_value / max_edge_metric
                scaled = np.sqrt(normalized)
                width = MIN_EDGE_WIDTH + (MAX_EDGE_WIDTH - MIN_EDGE_WIDTH) * scaled
            else:
                width = MIN_EDGE_WIDTH
            
            print(f"  {i+1}. Edge {edge_key}")
            print(f"     Weight: {edge_data['weight']:,.0f} tons, Value: ${edge_data['value']:,.0f}")
            print(f"     Metric value: {metric_value:,.0f}")
            print(f"     Normalized: {normalized:.4f} (= {metric_value:,.0f} / {max_edge_metric:,.0f})")
            print(f"     Scaled (sqrt): {scaled:.4f}")
            print(f"     WIDTH: {width:.4f}")
    else:
        print(f"No edges with traffic yet")

print(f"\n{'='*80}\n")

print(f"\nData processing complete:")
print(f"  Unique edges with traffic: {len(cumulative_edge_cargo)}")
print(f"  Total ships completed: {len(completed_ships)}")
print(f"  Max edge weight (at end of video): {max_edge_weight:,.0f} tons")
print(f"  Max edge value (at end of video): ${max_edge_value:,.0f}")
print()

# Create figure
fig, ax = plt.subplots(figsize=(20, 16), dpi=100)

def init():
    ax.clear()
    world.plot(ax=ax, color='lightgray', edgecolor='white', linewidth=0.5)
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)
    
    pos = nx.get_node_attributes(G_ch, 'pos')
    for (u, v) in G_ch.edges():
        x1, y1 = pos[u]
        x2, y2 = pos[v]
        ax.plot([x1, x2], [y1, y2], color='black', linewidth=MIN_EDGE_WIDTH,
               alpha=EDGE_NO_TRAFFIC_ALPHA, zorder=1)
    
    port_nodes = [n for n in G_ch.nodes() if G_ch.nodes[n].get('source') == 'port']
    port_pos = {n: pos[n] for n in port_nodes}
    nx.draw_networkx_nodes(G_ch, port_pos, nodelist=port_nodes,
                          node_color='green', node_size=150, alpha=0.5,
                          node_shape='s', ax=ax)
    
    ax.set_facecolor('#E8F4F8')
    ax.grid(True, linestyle='--', alpha=0.3)
    return []

pbar = tqdm(total=len(timesteps), desc="Rendering frames")

def animate(frame_idx):
    ax.clear()
    
    # Redraw base map
    world.plot(ax=ax, color='lightgray', edgecolor='white', linewidth=0.5)
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)
    
    # Get data for this frame
    frame_edge_cargo = edge_cargo_by_frame[frame_idx]
    frame_completed_cargo = completed_cargo_by_frame[frame_idx]
    
    # Use maximum from the FINAL frame (end of video) for scaling
    max_edge_metric = max_edge_weight if EDGE_METRIC == 'weight' else max_edge_value
    
    # Draw edges with varying widths based on CURRENT frame cumulative cargo
    pos = nx.get_node_attributes(G_ch, 'pos')
    
    for (u, v) in G_ch.edges():
        edge_key = (u, v)
        edge_data = frame_edge_cargo.get(edge_key, {'weight': 0, 'value': 0})
        metric_value = edge_data.get(EDGE_METRIC, 0)
        
        if metric_value > 0:
            if max_edge_metric > 0:
                normalized = metric_value / max_edge_metric
                scaled = np.sqrt(normalized)
                width = MIN_EDGE_WIDTH + (MAX_EDGE_WIDTH - MIN_EDGE_WIDTH) * scaled
            else:
                width = MIN_EDGE_WIDTH
            alpha = EDGE_WITH_TRAFFIC_ALPHA
        else:
            width = MIN_EDGE_WIDTH
            alpha = EDGE_NO_TRAFFIC_ALPHA
        
        x1, y1 = pos[u]
        x2, y2 = pos[v]
        ax.plot([x1, x2], [y1, y2], color='black', linewidth=width,
               alpha=alpha, zorder=1)
    
    # Draw ports
    port_nodes = [n for n in G_ch.nodes() if G_ch.nodes[n].get('source') == 'port']
    port_pos = {n: pos[n] for n in port_nodes}
    ports_by_color = {'green': [], 'orange': [], 'red': []}
    
    for node in port_nodes:
        port_name = G_ch.nodes[node].get('portName')
        num_ships, capacity = 0, 1
        if frame_idx in port_occupancy_by_timestep and port_name in port_occupancy_by_timestep[frame_idx]:
            num_ships, capacity = port_occupancy_by_timestep[frame_idx][port_name]
        
        color = get_port_color(num_ships, capacity)
        ports_by_color[color].append(node)
    
    for color, nodes in ports_by_color.items():
        if nodes:
            node_pos = {n: port_pos[n] for n in nodes}
            nx.draw_networkx_nodes(G_ch, node_pos, nodelist=nodes,
                                  node_color=color, node_size=150, alpha=0.5,
                                  node_shape='s', ax=ax)
    
    # Draw ships grouped by type with DIFFERENT SHAPES
    day, locations_data = timesteps[frame_idx]
    ships_by_type = {
        'cargo ship': {'positions': [], 'sizes': []},
        'tanker': {'positions': [], 'sizes': []},
        'bulk carrier': {'positions': [], 'sizes': []}
    }
    
    for ship_id, loc in locations_data.items():
        ship_id_int = int(ship_id)
        
        # Get position
        lon, lat = None, None
        
        if loc['status'] == 'active':
            edge = loc['edge']
            node1 = convert_node_id(edge[0])
            node2 = convert_node_id(edge[1])
            progress_fraction = loc['progress_fraction']
            
            try:
                lon1, lat1 = G_ch.nodes[node1]['lon'], G_ch.nodes[node1]['lat']
                lon2, lat2 = G_ch.nodes[node2]['lon'], G_ch.nodes[node2]['lat']
                lon = lon1 + progress_fraction * (lon2 - lon1)
                lat = lat1 + progress_fraction * (lat2 - lat1)
            except KeyError:
                continue
        
        elif loc['status'] in ['loading', 'unloading']:
            port_name = loc['port']
            if port_name in port_name_to_node:
                port_node = port_name_to_node[port_name]
                try:
                    lon = G_ch.nodes[port_node]['lon']
                    lat = G_ch.nodes[port_node]['lat']
                except KeyError:
                    continue
        
        if lon is None or lat is None:
            continue
        
        # Get ship data
        ship_type = ship_type_map.get(ship_id_int, 'cargo ship')
        weight = ship_weight_map.get(ship_id_int, 0)
        
        # Calculate size
        size = MIN_SHIP_SIZE + (MAX_SHIP_SIZE - MIN_SHIP_SIZE) * \
               ((weight - min_weight) / (max_weight - min_weight)) if max_weight > min_weight else MIN_SHIP_SIZE
        
        # Add to type group
        if ship_type in ships_by_type:
            ships_by_type[ship_type]['positions'].append((lon, lat))
            ships_by_type[ship_type]['sizes'].append(size)
    
    # Draw ships by type with DIFFERENT MARKERS
    # Cargo ship: circle (o), Tanker: square (s), Bulk carrier: triangle (^)
    if ships_by_type['cargo ship']['positions']:
        lons, lats = zip(*ships_by_type['cargo ship']['positions'])
        ax.scatter(lons, lats, c=SHIP_TYPE_COLORS['cargo ship']['color'],
                  s=ships_by_type['cargo ship']['sizes'], alpha=0.9,
                  edgecolors='black', linewidths=0.5, marker='o', zorder=10)
    
    if ships_by_type['tanker']['positions']:
        lons, lats = zip(*ships_by_type['tanker']['positions'])
        ax.scatter(lons, lats, c=SHIP_TYPE_COLORS['tanker']['color'],
                  s=ships_by_type['tanker']['sizes'], alpha=0.9,
                  edgecolors='black', linewidths=0.5, marker='s', zorder=10)
    
    if ships_by_type['bulk carrier']['positions']:
        lons, lats = zip(*ships_by_type['bulk carrier']['positions'])
        ax.scatter(lons, lats, c=SHIP_TYPE_COLORS['bulk carrier']['color'],
                  s=ships_by_type['bulk carrier']['sizes'], alpha=0.9,
                  edgecolors='black', linewidths=0.5, marker='^', zorder=10)
    
    ax.set_facecolor('#E8F4F8')
    ax.grid(True, linestyle='--', alpha=0.3)
    
    # ==============================================================================
    # TITLE: Simple day + progress bar
    # ==============================================================================
    progress_pct = (frame_idx + 1) / len(timesteps) * 100
    
    # Create progress bar using Unicode blocks
    bar_width = 40
    filled = int(bar_width * (frame_idx + 1) / len(timesteps))
    bar = '█' * filled + '░' * (bar_width - filled)
    
    title_text = f'Day {day:.1f}\n[{bar}] {progress_pct:.1f}%'
    ax.set_title(title_text, fontsize=16, fontweight='bold', pad=20)
    
    # ==============================================================================
    # TEXT BOX: Completed cargo ranking (TOP-LEFT)
    # ==============================================================================
    total_completed_weight = frame_completed_cargo['weight']
    total_completed_value = frame_completed_cargo['value']
    completed_hs_weights = frame_completed_cargo['hs_weights']
    completed_hs_values = frame_completed_cargo['hs_values']
    
    # Rank by chosen metric
    if EDGE_METRIC == 'weight':
        sorted_hs = sorted(completed_hs_weights.items(), key=lambda x: x[1], reverse=True)[:5]
    else:
        sorted_hs = sorted(completed_hs_values.items(), key=lambda x: x[1], reverse=True)[:5]
    
    # Build text box content
    textbox_lines = ['COMPLETED SHIPMENTS']
    textbox_lines.append(f'Weight: {total_completed_weight:,.0f} tons')
    textbox_lines.append(f'Value: ${total_completed_value:,.0f}')
    textbox_lines.append('')
    textbox_lines.append(f'Top 5 by {EDGE_METRIC}:')
    
    for hs_code, metric_val in sorted_hs:
        hs_str = str(hs_code).zfill(2)
        hs_name = hs_codes_mapping.get(hs_str, {}).get('name', f'HS{hs_str}')
        
        # Shorten name if too long
        if len(hs_name) > 25:
            hs_name = hs_name[:22] + '...'
        
        weight_val = completed_hs_weights.get(hs_code, 0)
        value_val = completed_hs_values.get(hs_code, 0)
        
        if EDGE_METRIC == 'weight':
            textbox_lines.append(f'{hs_name}: {weight_val:,.0f}t')
        else:
            textbox_lines.append(f'{hs_name}: ${value_val:,.0f}')
    
    textbox_text = '\n'.join(textbox_lines)
    
    # Add text box (top-left)
    ax.text(0.02, 0.98, textbox_text,
            transform=ax.transAxes,
            fontsize=10,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='black'))
    
    # ==============================================================================
    # LEGEND: Ship types by shape (BOTTOM-LEFT)
    # ==============================================================================
    legend_lines = ['SHIP TYPES']
    legend_lines.append('○ Cargo Ship')
    legend_lines.append('■ Tanker')
    legend_lines.append('▲ Bulk Carrier')
    
    legend_text = '\n'.join(legend_lines)
    
    # Add legend (bottom-left)
    ax.text(0.02, 0.02, legend_text,
            transform=ax.transAxes,
            fontsize=10,
            verticalalignment='bottom',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='black'))
    
    # Update progress
    pbar.update(1)
    
    return []

# Create animation
print(f"Building animation object ({FPS} fps)...")
anim = animation.FuncAnimation(fig, animate, init_func=init,
                              frames=len(timesteps),
                              interval=1000//FPS, blit=True)

# Save animation
print(f"\nSaving animation ({len(timesteps)} frames at {FPS} fps)...")
print(f"Video duration: {video_duration:.1f} seconds\n")

saved = False

# Try Pillow (GIF)
try:
    print("Attempting to save as GIF with Pillow...")
    from matplotlib.animation import PillowWriter
    writer = PillowWriter(fps=FPS)
    output_file = 'visualization_outputs/mediterranean_shipping_simulation.gif'
    anim.save(output_file, writer=writer, dpi=80)
    pbar.close()
    print(f"✓ Animation saved successfully as GIF: {output_file}")
    print(f"  Note: For better quality MP4, install ffmpeg")
    saved = True

except Exception as e:
    pbar.close()
    print(f"✗ Pillow GIF failed: {str(e)[:200]}")

# Try ffmpeg (MP4)
if not saved:
    try:
        print("\nAttempting to save as MP4 with ffmpeg...")
        pbar = tqdm(total=len(timesteps), desc="Rendering frames")
        
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=FPS, metadata=dict(artist='Mediterranean Shipping Simulation'),
                       bitrate=3000)
        
        output_file = 'visualization_outputs/mediterranean_shipping_simulation.mp4'
        anim.save(output_file, writer=writer, dpi=100)
        pbar.close()
        print(f"✓ Video saved successfully: {output_file}")
        saved = True
        
    except Exception as e:
        pbar.close()
        print(f"✗ ffmpeg not available: {str(e)[:200]}")
        print("  Install with: brew install ffmpeg")

plt.close(fig)
print("\nDone!")

Creating animated video visualization...
This may take several minutes...

Building canonical edge lookup...
Canonical edge lookup built: 1052 entries (526 unique edges)

Limiting animation to first 1000 frames (out of 8760 total)
Video duration: 50.0 seconds at 20 fps
Timestep range: Day 0.0 to Day 41.6

Pre-computing edge cargo accumulation and completed ships tracking...


Computing data:  54%|█████▍    | 543/1000 [00:00<00:00, 5421.54it/s]


FRAME 0 (Day 0.0)

FRAME 1 (Day 0.0)

FRAME 2 (Day 0.1)

FRAME 3 (Day 0.1)

FRAME 4 (Day 0.2)

FRAME 5 (Day 0.2)

FRAME 6 (Day 0.2)

FRAME 7 (Day 0.3)

FRAME 8 (Day 0.3)

FRAME 9 (Day 0.4)

FRAME 10 (Day 0.4)

FRAME 11 (Day 0.5)

FRAME 12 (Day 0.5)

FRAME 13 (Day 0.5)

FRAME 14 (Day 0.6)

FRAME 15 (Day 0.6)

FRAME 16 (Day 0.7)

FRAME 17 (Day 0.7)

FRAME 18 (Day 0.8)

Ship 1 entering edge (4425, 'port_34')
  Ship weight: 29,466 tons, Ship value: $14,987,222
  Edge weight BEFORE: 0 tons
  Edge value BEFORE:  $0
  Edge weight AFTER:  29,466 tons (+29,466)
  Edge value AFTER:   $14,987,222 (+$14,987,222)

FRAME 19 (Day 0.8)

Ship 7 entering edge (4821, 'port_24')
  Ship weight: 92,315 tons, Ship value: $30,927,263
  Edge weight BEFORE: 0 tons
  Edge value BEFORE:  $0
  Edge weight AFTER:  92,315 tons (+92,315)
  Edge value AFTER:   $30,927,263 (+$30,927,263)

FRAME 20 (Day 0.8)

Ship 1 entering edge (9629, 4425)
  Ship weight: 29,466 tons, Ship value: $14,987,222
  Edge weight BEFORE: 0 t

Computing data: 100%|██████████| 1000/1000 [00:00<00:00, 4647.79it/s]



FINAL MAX VALUES (from frame 999):
  Max edge weight: 10,808,106 tons
  Max edge value:  $13,192,390,855


EDGE WIDTH CALCULATIONS FOR FIRST 50 FRAMES:
Using EDGE_METRIC = 'weight'
MIN_EDGE_WIDTH = 0.3, MAX_EDGE_WIDTH = 10.0

FRAME 0 EDGE WIDTHS:
No edges with traffic yet

FRAME 1 EDGE WIDTHS:
No edges with traffic yet

FRAME 2 EDGE WIDTHS:
No edges with traffic yet

FRAME 3 EDGE WIDTHS:
No edges with traffic yet

FRAME 4 EDGE WIDTHS:
No edges with traffic yet

FRAME 5 EDGE WIDTHS:
No edges with traffic yet

FRAME 6 EDGE WIDTHS:
No edges with traffic yet

FRAME 7 EDGE WIDTHS:
No edges with traffic yet

FRAME 8 EDGE WIDTHS:
No edges with traffic yet

FRAME 9 EDGE WIDTHS:
No edges with traffic yet

FRAME 10 EDGE WIDTHS:
No edges with traffic yet

FRAME 11 EDGE WIDTHS:
No edges with traffic yet

FRAME 12 EDGE WIDTHS:
No edges with traffic yet

FRAME 13 EDGE WIDTHS:
No edges with traffic yet

FRAME 14 EDGE WIDTHS:
No edges with traffic yet

FRAME 15 EDGE WIDTHS:
No edges with traffic yet


Rendering frames:   0%|          | 0/1000 [00:00<?, ?it/s]

Building animation object (20 fps)...

Saving animation (1000 frames at 20 fps)...
Video duration: 50.0 seconds

Attempting to save as GIF with Pillow...


Rendering frames: 100%|██████████| 1000/1000 [15:15<00:00,  1.09it/s]

✓ Animation saved successfully as GIF: visualization_outputs/mediterranean_shipping_simulation.gif
  Note: For better quality MP4, install ffmpeg

Done!
